**This is the code of deep ANN to conduct prediction on the multi-scale Lorenz 96.There is no overfitting in the training phase because the final training and testing accuracies are the same. Our code is developed in Keras. It must be
noted that unlike RC-ESN (and RNN-LSTM),this ANN is stateless, i.e., there is no hidden variable such as states that tracks temporal evolution.**


In [ ]:
#importing all the required libraries.
import sys
import numpy as np
import scipy.sparse as sparse
from scipy.sparse import linalg
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import math
import keras
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import RMSprop, SGD, Adagrad, Adadelta
#import json
import matplotlib.pyplot as plt
from numpy import genfromtxt



In [ ]:
# loading lorenz data 
np.random.seed(42)
dataf = pd.read_csv("lorenz_data.csv",header=None)
data = np.array(dataf)
print(np.shape(data))


(1000000, 8)


In [ ]:
# Defining the global variables
shift_k = 0
shift_k=int(shift_k)
res_params = {
             'train_length': 50000,
             'predict_length': 2000
              }


In [ ]:
# training the  reservoir
train = data[shift_k:shift_k+res_params['train_length'],:]
label = data[1+shift_k:1+shift_k+res_params['train_length'],:]
print('np.shape(train)', np.shape(train))
print('np.shape(label)', np.shape(label))
print(train)
print(label)


np.shape(train) (50000, 8)
np.shape(label) (50000, 8)
[[-0.41182  -1.1352   -0.27178  ... -0.61869   0.083132 -1.4884  ]
 [-0.36659  -1.1066   -0.26038  ... -0.60283   0.096555 -1.4618  ]
 [-0.32292  -1.0767   -0.24848  ... -0.58697   0.10956  -1.4345  ]
 ...
 [-2.0074   -0.53604   1.263    ...  0.34168   1.7997   -1.0927  ]
 [-1.9443   -0.613     1.268    ...  0.3666    1.7756   -1.2379  ]
 [-1.8709   -0.69004   1.2693   ...  0.3915    1.7467   -1.3773  ]]
[[-0.36659  -1.1066   -0.26038  ... -0.60283   0.096555 -1.4618  ]
 [-0.32292  -1.0767   -0.24848  ... -0.58697   0.10956  -1.4345  ]
 [-0.28086  -1.0455   -0.23614  ... -0.5711    0.12218  -1.4065  ]
 ...
 [-1.9443   -0.613     1.268    ...  0.3666    1.7756   -1.2379  ]
 [-1.8709   -0.69004   1.2693   ...  0.3915    1.7467   -1.3773  ]
 [-1.7874   -0.76548   1.2674   ...  0.41625   1.713    -1.51    ]]


In [ ]:

y_train = label - train
x_train = train
print('np.shape(y_train)', np.shape(y_train))
print('np.shape(x_train)', np.shape(x_train))



np.shape(y_train) (50000, 8)
np.shape(x_train) (50000, 8)


In [ ]:
#building of neural network model
#activation function used here is tanh 
model = Sequential()
model.add(Dense(8, input_dim=8, activation='tanh'))
model.add(Dense(100, activation='tanh'))
model.add(Dense(100, activation='tanh'))
model.add(Dense(100, activation='tanh'))
model.add(Dense(100, activation='tanh'))
model.add(Dense(8, activation='tanh'))

#complilation of model
model.compile(loss='mean_absolute_error', optimizer='SGD', metrics=['mae'])

#fitting of the model
model.fit(x_train, y_train,epochs=200,batch_size=128,validation_split=0.2)

# weights of the network are computed using backpropagation optimized by the stochastic gradient descent algorithm
model.save_weights("./weights_Shift"+str(shift_k)+"K")



Epoch 1/200
313/313 [==============================] - 1s 3ms/step - loss: 0.0884 - mae: 0.0884 - val_loss: 0.0300 - val_mae: 0.0300
Epoch 2/200
313/313 [==============================] - 1s 2ms/step - loss: 0.0285 - mae: 0.0285 - val_loss: 0.0284 - val_mae: 0.0284
Epoch 3/200
313/313 [==============================] - 1s 2ms/step - loss: 0.0275 - mae: 0.0275 - val_loss: 0.0279 - val_mae: 0.0279
Epoch 4/200
313/313 [==============================] - 1s 2ms/step - loss: 0.0271 - mae: 0.0271 - val_loss: 0.0278 - val_mae: 0.0278
Epoch 5/200
313/313 [==============================] - 1s 2ms/step - loss: 0.0267 - mae: 0.0267 - val_loss: 0.0274 - val_mae: 0.0274
Epoch 6/200
313/313 [==============================] - 1s 2ms/step - loss: 0.0265 - mae: 0.0265 - val_loss: 0.0271 - val_mae: 0.0271
Epoch 7/200
313/313 [==============================] - 1s 2ms/step - loss: 0.0262 - mae: 0.0262 - val_loss: 0.0268 - val_mae: 0.0268
Epoch 8/200
313/313 [==============================] - 1s 2ms/step - 

In [ ]:
print ('Read reference state')
#reference state contains a reference to the previous time step for further prediction
# Load the reference state
ref_state = np.transpose(data[shift_k+res_params['train_length']:shift_k+res_params['train_length']+res_params['predict_length'],:])
print('np.shape(ref_state)',np.shape(ref_state))

train_y = data[1+shift_k:1+shift_k+res_params['train_length'],:]
print('np.shape(train_y)',np.shape(train_y))

n_dummy = np.shape(ref_state)
n_forecasts = 1
n_steps =  n_dummy[1]

#fore_state gives the starting point of time step for forecasting of the model
fore_state = np.zeros((n_forecasts*(n_steps+1),8))
state = np.zeros(8)
state_n = np.zeros((1,8))

out0 = np.zeros((8,1))                   #tn
out1 = np.zeros((8,1))                   #tn-1
out2 = np.zeros((8,1))                   #tn-2
out3 = np.zeros((8,1))                   #tn-3


Read reference state
np.shape(ref_state) (8, 2000)
np.shape(train_y) (50000, 8)


In [ ]:
# Get the last point from the training as the starting point of forcasting
state[:] = train_y[res_params['train_length']-1,:]
fore_state[0,:] = state[:]
# Adams-Bashforth integration scheme is used here for prediction
for j in range(n_steps):
    out3=out2
    out2=out1
    state_n[0,:] = state

    out1 = model.predict(state_n,batch_size=1)
    if j==0:
        out0 = out1                                                #initial value
    if j==1:
        out0 = 1.5*out1-0.5*out2                                   #two-step Adams-Bashforth method
    if j>1:
        out0 = (23.0/12.0)*out1-(4.0/3.0)*out2+(5.0/12.0)*out3     #three-step Adams-Bashforth method
    state[:] = state[:] + out0
    fore_state[1*(0)+j+1,:] = state[:]


In [ ]:

np.savetxt('output_ANN'+ 'shift'+str(shift_k)+ 'trainN' + str(res_params['train_length'])+'.csv',fore_state,delimiter=',')
np.savetxt('truth_ANN'+ 'shift'+str(shift_k)+ 'trainN' + str(res_params['train_length'])+'.csv',ref_state,delimiter=',')
print('done')

done
